In [2]:
import json
import os
import pandas as pd
import pyarrow as pq
import matplotlib.pyplot as plt
import numpy as np

In [2]:
jsons = []

In [3]:
for file in os.listdir("data/matches/raw/"):
    data = json.load(open("data/matches/raw/"+file,encoding="utf8"))
    jsons.append(data)

In [4]:
data = jsons[0]

In [5]:
data.keys()

dict_keys(['id', 'name', 'NameWithoutExtension', 'date', 'source', 'comment', 'status', 'client_name', 'hostname', 'type', 'tickrate', 'server_tickrate', 'duration', 'ticks', 'map_name', 'path', 'cheater_count', 'score_team1', 'score_team2', 'score_half1_team1', 'score_half1_team2', 'score_half2_team1', 'score_half2_team2', '5k_count', '4k_count', '3k_count', '2k_count', '1k_count', 'team_ct', 'team_t', 'team_surrender', 'team_winner', 'rounds', 'players', 'most_killing_weapon', 'overtimes', 'most_headshot_player', 'most_bomb_planted_player', 'most_entry_kill', 'bomb_planted', 'bomb_defused', 'bomb_exploded', 'kills', 'weapon_fired', 'player_blinded_events', 'player_hurted', 'kill_count', 'clutch_count', 'trade_kill_count', 'bomb_defused_count', 'bomb_planted_count', 'bomb_exploded_count', 'flashbang_thrown_count', 'smoke_thrown_count', 'he_thrown_count', 'decoy_thrown_count', 'molotov_thrown_count', 'incendiary_thrown_count', 'decoys', 'incendiaries', 'molotovs', 'damage_health_count'

In [13]:
#Transfer to SPARK to create players df
pd.DataFrame.from_records(data['rounds'])[['winner_name','winner_side','team_ct_name','team_t_name']]

,winner_name,winner_side,team_ct_name,team_t_name
0,FURIA eSports,CT,Evil Geniuses,FURIA eSports
1,FURIA eSports,CT,FURIA eSports,Evil Geniuses
2,FURIA eSports,CT,FURIA eSports,Evil Geniuses
3,FURIA eSports,CT,FURIA eSports,Evil Geniuses
4,Evil Geniuses,T,FURIA eSports,Evil Geniuses
5,FURIA eSports,CT,FURIA eSports,Evil Geniuses
6,FURIA eSports,CT,FURIA eSports,Evil Geniuses
7,FURIA eSports,CT,FURIA eSports,Evil Geniuses
8,FURIA eSports,CT,FURIA eSports,Evil Geniuses
9,Evil Geniuses,T,FURIA eSports,Evil Geniuses


In [29]:
x = pd.DataFrame.from_records(data['rounds'])[['winner_name','winner_side','team_ct_name','team_t_name']]

In [32]:
len(x)

25

In [31]:
len(np.where(x.winner_name == 'CT', x.team_ct_name, x.team_t_name))

25

In [11]:
#Transfer to SPARK to create players df
pd.json_normalize(pd.DataFrame.from_records(data['team_t'])['team_players']).head() 

,steamid,name,kill_count,crouch_kill_count,jump_kill_count,score,tk_count,assist_count,trade_kill_count,trade_death_count,...,time_death_rounds.16,time_death_rounds.17,time_death_rounds.18,time_death_rounds.19,time_death_rounds.20,time_death_rounds.21,time_death_rounds.22,time_death_rounds.23,time_death_rounds.24,time_death_rounds.25
0,76561198988083423,honda,16,2,0,40,0,4,4,2,...,63.18,96.82,37.73,71.56,103.07,76.63,62.83,0.00,47.45,0.00
1,76561197996370184,VINI,16,3,0,41,0,9,3,6,...,0.00,98.80,63.87,63.05,90.30,0.00,33.24,0.00,121.30,0.00
2,76561198058500492,KSCERATO,21,3,0,62,0,6,5,7,...,0.00,127.61,37.15,73.31,0.00,0.00,94.25,50.35,130.33,0.00
3,76561198043769572,arT,25,0,0,55,0,4,1,4,...,71.74,115.37,36.12,58.06,0.00,0.00,19.80,0.00,45.72,53.15
4,76561198164970560,yuurih,20,4,0,50,0,5,2,4,...,64.77,132.73,81.12,75.01,68.80,75.79,30.95,53.51,0.00,0.00


In [44]:
pd.json_normalize(x[x.columns[-1]]).columns[:20]

Index(['kd', 'mvp_count', 'hltv_rating', 'esea_rws', 'shot_count', 'hit_count',
       'is_vac_banned', 'is_ow_banned', 'flashbang_count', 'smoke_count'],
      dtype='object')